In [6]:
%run 02_data_preparation.ipynb

# Text representation methods

As Machine Learning methods take the data encoded as mathematical vectors, we need to convert our texts to this format. There is no just a single way of doing so, and we are now hoing to check different possibilities.

## Bag of words

The simplest way of encoding textual information in mathematical vector is to keep the information about the words which have been used. Typically, we firstly extract the list of all the unique words occurred in the dataset and assign each of them a number - a position in the dictionary which is then used as a position in the created vector describing the text. Let's consider the following example:

## TFIDF

## Word2vec

In [7]:
preprocess_text("Gdsagadsg")

'gdsagadsg'